In [1]:
%matplotlib inline

# Author: Xiang Wang

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms, models
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

## 一、文件夹结构设计和数据存放

### 1. 建立存放数据的文件夹

In [49]:
# shutil －一种高层次的文件操作工具，强大之处是在对文件的复制与删除操作支持比较好
import os
import shutil

In [ ]:
# 建立data文件夹用于存放数据，train和val分别存放训练集和验证集
# .mkdir的函数用于建立该路径下的文件夹
os.mkdir('/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data')
os.mkdir('/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data/train')
os.mkdir('/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data/val')

In [4]:
# 得到id和label，以便以下面分割数据集
import pandas as pd

idLabel = pd.read_csv('/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/zip/trainLabels.csv')

In [ ]:
# 创建训练集分类的label的文件夹
train_root = '/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data/train'

for label in idLabel.label.unique():
    os.mkdir(os.path.join(train_root, label))

In [ ]:
# 创建测试集分类的label的文件夹
val_root = '/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data/val'

for label in idLabel.label.unique():
    os.mkdir(os.path.join(val_root, label))

### 2. 分割数据集并将其存入对应文件夹


In [ ]:
# .listdir主要用于列出该路径下所有的文件的名称，以列表的形式得到
# 这里的路径是下载下来的CIFAR-10的训练集数据存放的路径
data_file = os.listdir('/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/zip/train')

In [ ]:
# 分割数据集
import numpy as np
idLabel_train = np.array(idLabel[['id','label']])[:48000]
idLabel_val = np.array(idLabel[['id','label']])[48000:]

In [ ]:
# 移动图片在训练集的相应位置
root = '/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/'

for (id,label) in idLabel_train:
    
    pic_path = root + 'zip/train/' + str(id) + '.png'
    
    obj_path = train_root + '/' + str(label) + '/' + str(id) + '.png'
    
    shutil.copy(pic_path, obj_path)

In [ ]:
# 移动图片到验证集的相应位置
root = '/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/'

for (id,label) in idLabel_val:
    
    pic_path = root + 'zip/train/' + str(id) + '.png'
    
    obj_path = val_root + '/' + str(label) + '/' + str(id) + '.png'
    
    shutil.copy(pic_path, obj_path)

## 二、数据处理与加载

In [12]:
print('==> Preparing data..')

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]),

    'val': transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]),
    'test': transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]),
}


==> Preparing data..


In [13]:
data_dir = '/Users/macbook/Documents/Programme/pytorch/Data/CIFAR-10/Data/'

image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) 
                  for x in ['train', 'val', 'test']}

# 得到可batch的数据格式
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=1)
               for x in ['train', 'val']}

testdataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=False, num_workers=1)
                   for x in ['test']}


# 得到数据规模
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val','test']}

# 得到分类名
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 三、建立模型

In [69]:
def alexNet_block(num_convs, in_channels, out_channels):
    
    net = [nn.Conv2d(in_channels, out_channels, kernel_size=5, stride=1, padding=0), nn.ReLU(True)]
    
    for i in range(num_convs-1):
        net.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1))
        net.append(nn.ReLU(True))
    
    net.append(nn.MaxPool2d(2,2))
    net.append(nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0))
    net.append(nn.MaxPool2d(kernel_size=2))

    return nn.Sequential(*net)
    

In [70]:
block_demo = alexNet_block(1, 3,6)
print(block_demo)

Sequential(
  (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [71]:
input_demo = Variable(torch.zeros(1, 3, 32, 32))
output_demo = block_demo(input_demo)
print(output_demo.shape)
np.shape(output_demo.view(output_demo.shape[0], -1))

torch.Size([1, 16, 5, 5])


torch.Size([1, 400])

In [169]:
class basicCNN(nn.Module):
    
    def __init__(self, num_classes=10):
        super(basicCNN, self).__init__()
        
        self.model_name = 'alexnet'
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(6, 16, kernel_size=5, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        
        
        self.fc = nn.Sequential(
            nn.Dropout(),
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(inplace=True),

            nn.Dropout(),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),

            nn.Linear(84, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.shape[0], -1)

        x = self.fc(x)
        return x

In [170]:
model_ft = basicCNN()

## 四、设置必要的参数

In [154]:
# 设置为GPU处理
model_ft = model_ft.to(device)

# 设置损失函数
criterion = nn.CrossEntropyLoss()
# 设置优化算法
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

## 五、训练阶段

In [171]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    # static_dict()用于返回包含模块所有状态的字典，包括参数和缓存。键是参数名称或者缓存名称。
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # 加入一个batch，将得到的数据放置到制定的设备上（GPU或者CPU）
            for inputs, labels in dataloaders[phase]:
                inputs = inputs
                labels = labels

                # 清空梯度
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                # 以前是使用.data[0]来得到loss，不过官方推荐使用item来转化这里数据
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [173]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=2)

Epoch 0/1
----------
train Loss: 2.3093 Acc: 0.0973
val Loss: 2.3100 Acc: 0.0820

Epoch 1/1
----------
train Loss: 2.3092 Acc: 0.0975
val Loss: 2.3107 Acc: 0.0820

Training complete in 2m 34s
Best val Acc: 0.082000
